In [ ]:
import requests
import matplotlib.pyplot as plt 
from PIL import Image
from io import BytesIO
import pandas as pd 
from datetime import datetime, timedelta
import datetime
import pytz
import matplotlib.pyplot as plt
#from ratelimit import limits

In [ ]:
import tensorflow as tf


if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is NOT available')

print(tf.__version__)

In [ ]:
import os
from io import BytesIO
from skimage import io
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import cartopy.crs as ccrs
#import cartopy
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import urllib.request
import urllib.parse
import mapbox_vector_tile
import xml.etree.ElementTree as xmlet
import lxml.etree as xmltree
from PIL import Image as plimg
import numpy as np
from owslib.wms import WebMapService
from IPython.display import Image, display
%matplotlib inline

In [ ]:
def convert_to_cet(timestamp : int):
    # Convert timestamp to UTC datetime object
    datetime_utc = datetime.datetime.utcfromtimestamp(int(timestamp))

    # Define UTC timezone
    utc_timezone = pytz.timezone('UTC')

    # Make datetime_utc aware of UTC timezone
    datetime_utc = utc_timezone.localize(datetime_utc)

    # Define CET timezone
    cet_timezone = pytz.timezone('CET')

    # Convert datetime from UTC to CET
    datetime_cet = datetime_utc.astimezone(cet_timezone)
    
    return datetime_cet.strftime('%H:%M:%S')

In [ ]:
#url = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day,Reference_Features_15m&CRS=EPSG:4326&TIME=2023-07-19&WRAP=DAY,DAY,X&BBOX=34.4682,19.1958,42.0973,28.6705&FORMAT=image/jpeg&WIDTH=216&HEIGHT=174&AUTOSCALE=TRUE&ts=1707854243677'


def get_url(date):
    return 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/7a06ce98ec9ffa8870664c552b909fcc/MODIS_NRT/19.382,34.8,29.667,44/1/'+date

def get_firms_modis_data(dateFrom, dateTo,file_save = False): 
    #greece 19.382,34.8,29.667,44
    
    
    API_KEY = ''
    headers = {'Authorization':API_KEY}
    resulting_df = pd.DataFrame()
    
    date = dateFrom
    while date <= dateTo:
        response = requests.get(get_url(date.strftime('%Y-%m-%d')),headers = headers)
        if response.status_code == 200:
            # Convert the response content into an image
            #img = Image.open(BytesIO(response.content))
            df = pd.read_csv(BytesIO(response.content))
            
            resulting_df = pd.concat([resulting_df,df],axis = 0)
            
            date += timedelta(days=1)
        else:
            print(f"Failed to retrieve the image for date {date}. Status code:", response.status_code)
            date += timedelta(days=1)
            continue
        
    resulting_df['acq_time'] = resulting_df['acq_time'].apply(convert_to_cet)
    
    if file_save:
        resulting_df.to_csv(f"Modis_data_Greece_{dateFrom.strftime('%Y-%m-%d')}_to_{dateTo.strftime('%Y-%m-%d')}.csv")
        
    return resulting_df

In [ ]:
date_start = datetime.datetime(2023,7,1)
date_end = datetime.datetime(2023,8,1)
#modis_data = get_firms_modis_data(date_start,date_end, file_save = True)
#modis_data

In [ ]:
modis_df = pd.read_csv('Modis_data_Greece_2023-07-01_to_2023-08-01.csv')
modis_df

In [ ]:
modis_df['daynight'].value_counts()

In [ ]:
plt.hist(modis_df['confidence'], edgecolor='black')
plt.title('Confidence Score')
plt.xlabel('confidence')
plt.ylabel('Number of incidents')

In [ ]:
long = modis_df['longitude']
lat = modis_df['latitude']

wildfire_locations = list(zip(lat,long))

# Create a map centered around the first wildfire location
map_center = wildfire_locations[0]
my_map = folium.Map(location=map_center, zoom_start=4)

# Add markers for each wildfire location
for location in wildfire_locations:
    folium.Marker(location=location, popup="Wildfire Location", icon=folium.Icon(color='red')).add_to(my_map)

# Save the map to an HTML file
my_map.save("wildfire_map.html")

### Weather Data

In [ ]:
import os
import pandas as pd
import datetime
import pytz
import requests
from io import BytesIO
#from ratelimit import sleep_and_retry, limits

In [ ]:
modis_path = './Modis 2017-2023'
virs_path = "./Virss-snpp 2017-2023"
print(os.listdir(modis_path)) 

In [ ]:
virs_df = pd.DataFrame()

for i in os.listdir(virs_path):
    df = pd.read_csv(virs_path+'/'+i)
    if i == 'viirs-snpp_2021_Greece.csv':
        df['acq_date'] = pd.to_datetime(df['acq_date'])
        df['acq_date'] = df['acq_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df['acq_time'] = df['acq_time'].apply(convert_to_cet)
    virs_df = pd.concat([virs_df,df],axis = 0)



In [ ]:
virs_dates = list(virs_df['acq_date'].unique())
virs_dates[-1]

In [ ]:
url = 'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2021-12-30&end_date=2021-12-30&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,wind_speed_10m_max'

r = requests.get(url)

data = r.json()['daily']
data

In [ ]:
r.json()

In [ ]:
long = virs_df['longitude']
lat = virs_df['latitude']
date = virs_df['acq_date']

In [ ]:
len(long), len(lat), len(date)  

In [ ]:
print(long.iloc[0],lat.iloc[0],date.iloc[0])

In [ ]:
missing_weather = pd.read_csv('missing_weather.csv')
miss_1 = missing_weather[missing_weather.isnull().any(axis=1)]
miss_1#'latitude','longitude','acq_date'

In [ ]:
long = miss_1['longitude']
lat = miss_1['latitude']
date = miss_1['acq_date']

In [ ]:


final_df = pd.DataFrame()


# @sleep_and_retry
# @limits(calls=9990, period=600)
def getWeatherData(lat,lon,date, final_df):
    try:
        url = f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,wind_speed_10m_max'

        r = requests.get(url)
        
        if 'daily' not in r.json().keys():
            return final_df
        data = r.json()['daily']

        lon_response = r.json()['longitude']
        lat_response = r.json()['latitude']
        df = pd.DataFrame(data)

        df['latitude'] = lat_response
        df['longitude'] = lon_response
        
        final_df = pd.concat([final_df,df],axis = 0)
    except Exception as e:
        final_df.to_csv('missing_weather_data.csv')
        raise e
    return final_df


In [ ]:
for i,j,k in zip(lat,long,date):
    
    final_df = getWeatherData(i,j,k,final_df)

final_df.to_csv('missing_weather_data.csv')


final_df
#getWeatherData(lat,long,date)

In [ ]:
dat = getWeatherData(40.636848,22.605183,'2017-01-01',final_df)
dat

In [ ]:
df1 = pd.read_csv('weather_2017-01-01_2019-04-01.csv')
time = df1[['time']]
time.rename(columns = {'time':'acq_date'},inplace=True)
time

In [ ]:
virs_df

In [ ]:
time2 = pd.Series(time['acq_date'])
time2

In [ ]:
time

In [ ]:
virs_df.index = virs_df['acq_date']
virs2 = virs_df[-virs_df['acq_date'].isin(time2)]

In [ ]:
lat_2 = virs2['latitude']
lon_2 = virs2['longitude']
date_2 = virs2['acq_date']

In [ ]:
time

In [ ]:
import pandas as pd
df1 = pd.read_csv(r')
time2 = df1[['time']]
time2.rename(columns = {'time':'acq_date'} ,inplace=True)
time2

In [ ]:
time2

In [ ]:
virs_df.index = virs_df['acq_date']
virs2 = virs_df[~virs_df.index.isin(time)]

In [ ]:
time2

In [ ]:
virs_df.index = virs_df['acq_date']
virs_df

In [ ]:
virs_df

In [ ]:
t = virs_df.loc[virs_df.index >= pd.Timestamp('2021-10-01')]
t

In [ ]:
lat_4 = t['latitude']
lon_4 = t['longitude']
date_4 = t['acq_date']

### NDVI Copernicus

In [ ]:
import requests
import pandas as pd
import json
import jwt
import time
import io

In [ ]:
dataset_id = '5d5f72ce-80bc-4c90-80ed-f135596533e2'
token = ''
ndvi_req = requests.get(f'https://land.copernicus.eu/@@oauth2-token?dataset_uid={dataset_id}&download_information_id=2c51dd60-0e70-4ff8-88c0-bd8cb6c58123', 
             headers={'Accept': 'application/json', 'Authorization': f'{token}'})


ndvi_req.content

In [ ]:
private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

In [ ]:
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
base_url = 'https://land.copernicus.eu'

dataset_id = '5d5f72ce-80bc-4c90-80ed-f135596533e2'

url_download_prepackaged = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [
    {
     'DatasetID': 'af077469527848f7a0fc8b220589fef1',
     'FileId': 'c20db9cf-cb81-4b87-be26-7d5bf224d7f8'
    }
]}

#print(url_download_prepackaged)
response_download_prepackaged = requests.post(url_download_prepackaged, headers=headers, json=data)
response_download_prepackaged.json()

In [ ]:
url_downloadable_prepackaged = f"{base_url}/api/@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=downloadable_files"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_downloadable_prepackaged = requests.get(url_downloadable_prepackaged, headers=headers)

json_downloadable_prepackaged = response_downloadable_prepackaged.json()

ndvi_global_dataset = 'https://land.copernicus.eu/api/en/products/vegetation/normalised-difference-vegetation-short-term-statistics-v3-0-1km'

for item in json_downloadable_prepackaged.get('items', []):
    print(item['@id'])
    # if item['@id'] == ndvi_global_dataset:
    #     print('Product information:')
    #     print('\n')
    #     print(3*' '+'Product title: "'+ item['title'].replace(':','')+'"')
    #     print(3*' '+'UID: "'+item['UID'].replace(' ','')+'"')
    #     print(3*' '+'Product link: '+ item['@id']+'')
    #     print('\n')
        
    #     for downloadable_file in item.get('downloadable_files', {}).get('items', []):
    #         print('Download option for "'+item['title'].replace(':','')+'":')
    #         display(downloadable_file)
    #         print(2*'\n')

    #     print('\n')
    #     print(100*'_')
    #     break

### Elevation & Slope

In [ ]:
import requests
import pandas as pd 

In [ ]:
merged_df = pd.read_csv(r'C:\Users\johnp\OneDrive\Υπολογιστής\IHU\Thesis\merged_data.csv')
missing_elevation =merged_df[['elevation','longitude','latitude']]

In [ ]:
miss_el = missing_elevation[missing_elevation.isnull().any(axis=1)]
miss_el


In [ ]:
long = miss_el['longitude']
lat = miss_el['latitude']

In [ ]:
long_lat = list(zip(lat,long))
lat_long_df = pd.DataFrame(long_lat,columns = ['latitude','longitude'])

In [ ]:
lat_long_df

In [ ]:
unique_coords = df[['latitude', 'longitude']].drop_duplicates().values.tolist()
len(unique_coords)


In [ ]:
pd.DataFrame(unique_coords, columns=['latitude', 'longitude']).to_csv('unique_coords.csv', index=False)

In [ ]:
unique_coords[0][0]

In [ ]:

def getElevationData(latitude, longitude, data_list):
    
    try:
        url = f'https://api.open-meteo.com/v1/elevation?latitude={latitude}&longitude={longitude}'

        r = requests.get(url)

        data = r.json()
        data_list.append(data['elevation'][0])
        
    except Exception as e:
        df = pd.DataFrame(data_list)
        df.to_csv('missing_elevation.csv')
        print(e)

In [ ]:
el_1 = pd.read_csv('elevation.csv')
el_1_count = el_1.shape[0]
el_1_count

In [ ]:
unique_coords_2 = unique_coords[el_1_count:]
len(unique_coords_2) 

In [ ]:
elevation_list = []

for x,y in zip(lat,long):
    getElevationData(x,y, elevation_list)

elevation_df = pd.DataFrame(elevation_list)

elevation_df.to_csv('missing_elevation.csv')

In [ ]:
elevation_df.shape[0] + el_1.shape[0]

In [ ]:
missing_slope = merged_df[['longitude','latitude','slope','aspect']]
miss_3 = missing_slope[missing_slope.isnull().any(axis=1)]
miss_3.to_csv('missing_slope.csv',index=False)

In [ ]:
elevation_df = pd.DataFrame(elevation_list)

elevation_df.to_csv('elevation.csv')